## Transfer Learning usando VGG16

Usando Transfer Learning, buscamos utilizar un modelo preentrenado para resolver un problema diferente al que fue entrenado originalmente. En este caso, utilizaremos la red VGG16 en el dataset sobre imagenes de animales, un dataset bastante pequeño, para comprobar los resultados que nos dan.

La principal ventaja que nos aporta VGG16, es que como ya tiene los pesos ajustado es una gran herramientas para extraer características de las imagenes.

## Código
### Librerías
Las librerías que utilizaremos son las siguientes:
- numpy: para el manejo de matrices
- torch.nn: para la creación de la red neuronal
- torch.optim: para la optimización de la red
- torchvision import models: para la importación de la red VGG16
- torchvision import datasets: para la importación del dataset
- torchvision import transforms: para la transformación de las imagenes
- torch.utils.data import DataLoader: para la creación de los dataloaders
- torch.utils.data import random_split: para la división del dataset en entrenamiento y test


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
import numpy as np

#### Comprobación de la GPU
Comprobamos si tenemos una GPU disponible para acelerar el entrenamiento de la red neuronal.

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Carga de datos
Primero adataremos los transformadores para las imagenes. Luego cargaremos el dataset de animales y crearemos los dataloaders para el entrenamiento y test.


In [3]:
# Transformaciones para redimensionar y normalizar las imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),    # Redimensionar a 224x224
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )
])

In [4]:
# Cargar el dataset completo
dataset = datasets.ImageFolder(root='dataset_animales/animals/animals/', transform=transform)

# Calcular las longitudes para entrenamiento y prueba (80% y 20%)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Dividir el dataset en entrenamiento y prueba
train_data, test_data = random_split(dataset, [train_size, test_size])

# Crear DataLoaders para el conjunto de entrenamiento y prueba
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Verificación de tamaños
print("Tamaño del conjunto de entrenamiento:", len(train_loader.dataset))
print("Tamaño del conjunto de prueba:", len(test_loader.dataset))

Tamaño del conjunto de entrenamiento: 336
Tamaño del conjunto de prueba: 85


### Carga de modelo VGG16
Cargaremos el modelo VGG16 preetrenado y cambiaremos la última capa para adaptarla a nuestro problema. Determinaremos la cantidad de clases que tiene nuestro problema y usando model.classifier[6] = nn.Linear(4096, num_classes) cambiaremos la última capa de la red de la parte de classifier.

Como conocemos la estructura de la red solo necesitaremos cambiar la última capa de la parte de clasificación ya que Torch nos permite acceder a las capas de la red de manera sencilla.

Si quisieramos cambiar las partes convolucionales de la red, tendríamos que cambiar la parte de features.

In [5]:
# Cargar el modelo VGG16 preentrenado
model = models.vgg16(pretrained=True)

# Cambiar la última capa fully connected para que coincida con el número de clases de tu dataset
num_classes = len(dataset.classes)
model.classifier[6] = nn.Linear(4096, num_classes)

c:\Users\Usuario\Documents\Ingenieria de Datos\3_AÑO\1_Semestre\AA2\Practicas\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Usuario\Documents\Ingenieria de Datos\3_AÑO\1_Semestre\AA2\Practicas\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


En la siguiente imagen se muestra la estructura de la red VGG16:

![VGG16](illu_VGG-02.png)

#### Moviendo el modelo a la GPU
Movemos el modelo a la GPU si está disponible.

In [6]:
# Mover el modelo a la GPU si está disponible
model = model.to(device)

### Entrenamiento y Test
Entrenaremos el modelo con un optimizador que será SGD y una función de pérdida que será CrossEntropyLoss.

El SGD usará un learning rate de 0.001 y un momentum de 0.9. También, tomaremos los parametros del modelo y los pasaremos al optimizador usando model.parameters().

In [7]:
# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.005)

# Entrenamiento del modelo
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # Evaluación en el conjunto de prueba
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy on the test set: {100 * correct / total:.2f}%')

Epoch [1/3], Loss: 1.7512
Accuracy on the test set: 100.00%
Epoch [2/3], Loss: 0.4545
Accuracy on the test set: 98.82%
Epoch [3/3], Loss: 0.0622
Accuracy on the test set: 97.65%


## Futuras Mejoras
Utilizar la congelación de las capas de feuatures (capas convolucionales) para que no se actualicen los pesos de estas capas y solo se actualicen los pesos de las capas de clasificación.

Esto ayudaría a utilizar la red preentrenada para extraer mejores las caracteristicas de las imagen y solo entrenar las capas de clasificación.